In [1]:
import os
import pandas as pd
import datetime
import numpy as np
import xlsxwriter
import asyncio
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import *
import datetime
from openpyxl import load_workbook , Workbook
from utils import staff_code

In [2]:
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
font1 = Font(name='Arial', size=10,  bold=False, italic=False,  vertAlign=None, underline='none', strike=False, color='FF000000')
font2 = Font(name='Arial', size=10,  bold=True, italic=False,  vertAlign=None, underline='none', strike=False, color='FF000000')
font3 = Font(name='Arial', size=12,  bold=True, italic=False,  vertAlign=None, underline='none', strike=False, color='FF000000')
font4 = Font(name='Arial', size=14,  bold=True, italic=False,  vertAlign=None, underline='none', strike=False, color='FF000000')

fill1 = PatternFill(fill_type='solid',start_color='FFC0C0C0', end_color='FFC0C0C0')

fill = PatternFill(fill_type=None,start_color='FFFFFFFF', end_color='FF000000')
border = Border(left=Side(border_style=None, color='FF000000'),
                right=Side(border_style=None, color='FF000000'),
                top=Side(border_style=None, color='FF000000'),
                bottom=Side(border_style=None, color='FF000000'),
                diagonal=Side(border_style=None, color='FF000000'),
                diagonal_direction=0,
                outline=Side(border_style=None, color='FF000000'),
                vertical=Side(border_style=None, color='FF000000'),
                horizontal=Side(border_style=None, color='FF000000')
)
border1 = Border(left=Side(border_style='thin', color='FF000000'),
                right=Side(border_style='thin', color='FF000000'),
                top=Side(border_style='thin', color='FF000000'),
                bottom=Side(border_style='thin', color='FF000000'),)
border2 = Border(bottom=Side(border_style='thin', color='FF000000'))
alignment1=Alignment(horizontal='center',vertical='center',text_rotation=0, wrap_text=True, shrink_to_fit=False, indent=0)

In [37]:
# Dado el nombre de un staff revisa todos los timesheet y crea un df con todos los horarios por proyecto
#
def weeks_hours(staff):
    path = os.path.join('O:','Libovich','F','Estudio','Time Sheets')
    files_dirs = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.xlsx') and staff in file and '2020' in file:
                print(root)
                files_dirs.append(os.path.join(root, file))
    columns_df = ['Date','In','Out','Hours','Project','Description']
    columns_tempdf = ['Date','Vacio1','In','Out','Hours','Project','Description','Vacio2']
    df = pd.DataFrame(columns = columns_df) 
    for file in files_dirs:
        try :
            temp_df = pd.read_excel(file, sheet_name ='Time Sheet', dtype=str )
            temp_df = temp_df.iloc[14: , 1:9]
            temp_df.columns = columns_tempdf
            temp_df.dropna(subset=['Project'], inplace = True)
            temp_df.dropna(1, inplace = True)
            if temp_df.shape[0] != 0:  
                df = df.append(temp_df, ignore_index = True)
        except:
            print('')
    df['Hours'] = df['Hours'].astype(float).round(2)
    df['Project'] = df['Project'].str.strip()
    del df['In']
    del df['Out']
    project_df = pd.read_excel(os.path.join(path,'Project Board.xls'))
    project_df = project_df.iloc[7: , :5]
    project_df.columns = ['CODE','CLIENT','Project','CODIGO EXTERNO','PROJECT NAME + CODIGO EXTERNO']
    project_df['Project'] = project_df['Project'].str.strip()
    df = pd.merge(df, project_df[['CODE','CLIENT','Project']],on='Project')
    sort_df = ['Date','CODE','CLIENT','Project','Hours','Description']
    df = df[['Date','CODE','CLIENT','Project','Hours','Description']]
    del df['CODE']
    return df

In [38]:
# Carga un resumen de horarios y devuelve el df
# 
def load_resumen(path):
    if os.path.isfile(path) :
        resumen_df = pd.read_excel(path, sheet_name ='Resumen')
        resumen_df.columns = resumen_df.iloc[3]
        resumen_df = resumen_df[4:]
        resumen_df.drop(['Fecha procesamiento','Valor hora','Horas por mes', 'Total', 'Pagos / Anticipos', 'Saldo'], axis = 1, inplace= True)
        return resumen_df
    else:
        with output2:
            print('Resumen no encontrado')
            print('Cargue el resumen primero')

In [75]:
# Toma el df de resumen y de los timesheet 
# y devuelve un df procesado sin las copias y de los valores
#
def process_df(df, resumen_df):
    if resumen_df.shape[0] != 0:
        append_df = df[~df.apply(tuple,1).isin(resumen_df.apply(tuple,1))].reset_index(drop=True)
        append_df.insert(loc=0, column='Fecha procesamiento', value= datetime.date.today().strftime('%d/%m/%Y')) 
        del append_df['Description']
        return append_df
    else:
        df.insert(loc=0, column='Fecha procesamiento', value= datetime.date.today().strftime('%d/%m/%Y')) 
        del df['Description']
        return df


In [76]:
# Dado un df, y el path a donde se encuentra el archivo para sumar las horas
# Agrega las horas trabajadas al excel
# Con las columnas de la funcion que se agrega
def append_excel(df, resumen_df, path):
    # for a given df of a month write that in the excel with values
    # Start by opening the spreadsheet and selecting the main sheet
    workbook = load_workbook(filename=path)
    sheet = workbook.active

    for r in dataframe_to_rows(df, index=False, header=False):
        sheet.append(r)
    for i in range(df.shape[0]):
        row_numb = str(resumen_df.shape[0]+i+6)
        #Total
        sheet['G' + row_numb] = '=F' +row_numb + '*E'+ row_numb
        #Saldo
        if int(row_numb) == 6:
            sheet['I' + row_numb] = '=G' + row_numb + '-H' + row_numb
        else:
            sheet['I' + row_numb] = '=G' + row_numb + '-H' + row_numb + '+I' + str(int(row_numb) - 1)
    
    # sheet['H' + row_numb] =  pago
    workbook.save(filename=path)

In [77]:
# Toma un df con la columna 'Mes' y dado un mes 
# devuelve el df del correspondiente mes sin la columna 'Mes'
# para llevar al excel

def month_df(df, month):
    month_df = df.loc[df['Mes'] == month]
    del month_df['Mes']
    month_df['Valor Hora'] = month
    return month_df

# Cargar time sheet en el Resumen
### Elegir el archivo resumen del staff

In [78]:
def main_func(b):
    staff = staff_picker.value
    file_name = 'Resumen '+staff+'.xlsx'
    folder_path = os.path.join('O:','Libovich','F','Estudio','Resumen')
    fname = os.path.join(folder_path , file_name)
    if not os.path.isfile(fname) : 
        create_resumen(staff , fname)
        
    resume_df = load_resumen(fname)
    append_df = process_df(weeks_hours(staff+'-'), resume_df)
    append_excel(append_df,resume_df,fname)

    with output:
        print('Se cargaron los horarios de ' + staff)

In [79]:
output = widgets.Output()

In [80]:
def create_resumen(staff , path):
    wb = Workbook()
    ws =  wb.active
    anchos = [10.71,10.71,10.71,10.71,10.71,10.71,11.71,10.71]
    for i , ancho in enumerate(anchos):
        ws.column_dimensions[get_column_letter(i+1)].width = ancho
    ws.row_dimensions[5].height = 25.5
    
    ws.title = "Resumen"
    ws['A1'] = staff[:2]
    ws['B1'] = staff[3:]
    ws['A3'] = 'Planilla individual de horas'
    ws['A5'] = 'Fecha procesamiento'
    ws['B5'] = 'Dia'
    ws['C5'] = 'Cliente'
    ws['D5'] = 'Proyecto'
    ws['E5'] = 'Cantidad de horas'
    ws['F5'] = 'Valor hora'
    ws['G5'] = 'Total'
    ws['H5'] = 'Pagos / Anticipos'
    ws['I5'] = 'Saldo'
    ws['J5'] = 'Horas por mes'
    
    ws['A1'].font = font4
    ws['B1'].font = font4
    ws['A3'].font = font4
    letter_tittle =['A', 'B','C','D','E','F','G','H','I','J']
    for letter in letter_tittle:
        ws[letter + '5'].font = font2
        ws[letter + '5'].fill = fill1
        ws[letter + '5'].border = border1
        ws[letter + '5'].alignment = alignment1
    wb.save(filename = path)

In [81]:
staffs = ['-'.join(i) for i in staff_code()] 
staff_picker = widgets.Dropdown(
    options=staffs,
    description='Staff',
    disabled=False,
)

button = widgets.Button(description="Cargar time sheets")
button.on_click(main_func)
button

widgets.HBox([
    staff_picker,
    button
    ],
    layout = widgets.Layout(width = '900px')
)

O:Libovich\F\Estudio\Time Sheets\2020-0101-0105
O:Libovich\F\Estudio\Time Sheets\2020-0101-0131-NO USAR\2020-0101-0105
O:Libovich\F\Estudio\Time Sheets\2020-0101-0131-NO USAR\2020-0106-0112
O:Libovich\F\Estudio\Time Sheets\2020-0101-0131-NO USAR\2020-0113-0119
O:Libovich\F\Estudio\Time Sheets\2020-0101-0131-NO USAR\2020-0120-0126
O:Libovich\F\Estudio\Time Sheets\2020-0101-0131-NO USAR\2020-0127-0131
O:Libovich\F\Estudio\Time Sheets\2020-0106-0112
O:Libovich\F\Estudio\Time Sheets\2020-0113-0119
O:Libovich\F\Estudio\Time Sheets\2020-0120-0126
O:Libovich\F\Estudio\Time Sheets\2020-0127-0131
O:Libovich\F\Estudio\Time Sheets\2020-0201-0202
O:Libovich\F\Estudio\Time Sheets\2020-0203-0209
O:Libovich\F\Estudio\Time Sheets\2020-0210-0216
O:Libovich\F\Estudio\Time Sheets\2020-0217-0223
O:Libovich\F\Estudio\Time Sheets\2020-0224-0229
O:Libovich\F\Estudio\Time Sheets\2020-0301-0301
O:Libovich\F\Estudio\Time Sheets\2020-0302-0308
O:Libovich\F\Estudio\Time Sheets\2020-0309-0315
O:Libovich\F\Estudio\

In [12]:
display(output)

Output()

# Create invoice

### Elegir el arquitecto para generar el recibo honorarios
### Generar para el corriente mes o para el mes anterior

In [13]:
def get_resumen_by_month(df, month):
    dates = pd.to_datetime(df['Dia'], format='%m/%d/%Y', errors='coerce')
    df['Mes'] = dates.dt.strftime('%Y-%m')
    return df.loc[df['Mes'] == month]

def get_this_month(b):
    today = datetime.date.today()
    add_recibo(staff_picker.value , today.strftime('%Y-%m'))

def get_last_month(b):
    last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)
    add_recibo(staff_picker.value , last_month.strftime('%Y-%m'))

In [14]:
def add_recibo (staff, month):
    file_name = 'Resumen '+staff+'.xlsx'
    folder_path = os.path.join('O:','Libovich','F','Estudio','Resumen')
    fname = os.path.join(folder_path , file_name)
    df = get_resumen_by_month(load_resumen(fname),month)
    if df.shape[0] != 0 : 
        df['Dia'] = pd.to_datetime(df['Dia'])
        df['Semana'] = df['Dia'].dt.strftime('%W').astype(int) +1 
        df = df.groupby(['Cliente','Proyecto','Semana', 'Mes']).sum().reset_index()
        df = df[['Semana', 'Mes','Cliente','Proyecto', 'Cantidad de horas']].sort_values('Semana').reset_index(drop = True)
        df.insert(loc=0, column='Periodo', value= df['Mes'].map(str) + '/' + df['Semana'].map(str))
        month = pd.to_datetime(df['Mes'], format='%Y-%m', errors='coerce')
        meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
        anchos = [2.71,20.43,8.71,18.86, 13.71,11.71,11.71,10.71]

        mes = month.iloc[0]

        wb2 = load_workbook(fname)
        sheet_recibo = wb2.create_sheet('Recibo '+ meses[mes.month-1])

        for i in range(7):
            sheet_recibo.column_dimensions[get_column_letter(i+1)].width = anchos[i]
        sheet_recibo['F2'] = 'Córdoba,'
        sheet_recibo['G2'] = meses[mes.month-1] +' '+ str(mes.year)
        sheet_recibo['B4'] = 'R E C I B I   de '
        sheet_recibo['C4'] = 'FXCollaborative'
        sheet_recibo['B6'] = 'la cantidad de PESOS'
        sheet_recibo['B9'] = 'Por honorarios profesionales correspondientes a los períodos y conceptos que a continuación se detallan:'
        sheet_recibo['B11'] ='Semana y Mes'
        sheet_recibo['C11'] ='Proyecto'
        sheet_recibo['E11'] ='Cantidad horas'
        sheet_recibo['F11'] ='Valor hora ($)'
        sheet_recibo['G11'] ='Total ($)'

        sheet_recibo['F2'].font = font1
        sheet_recibo['G2'].font = font1
        sheet_recibo['B4'].font = font1
        sheet_recibo['C4'].font = font2
        sheet_recibo['B6'].font = font1
        sheet_recibo['B9'].font = font1
        sheet_recibo['B11'].font = font1
        sheet_recibo['C11'].font = font1
        sheet_recibo['E11'].font = font1
        sheet_recibo['F11'].font = font1
        sheet_recibo['G11'].font = font1

        sheet_recibo['B11'].alignment = alignment1
        sheet_recibo['C11'].alignment = alignment1
        sheet_recibo['E11'].alignment = alignment1
        sheet_recibo['F11'].alignment = alignment1
        sheet_recibo['G11'].alignment = alignment1

        sheet_recibo['B11'].border = border1
        sheet_recibo['C11'].border = border1
        sheet_recibo['E11'].border = border1
        sheet_recibo['F11'].border = border1
        sheet_recibo['G11'].border = border1

        sheet_recibo['C6'].fill = fill1
        sheet_recibo['B7'].fill = fill1

        sheet_recibo.merge_cells('C11:D11')
        sheet_recibo.merge_cells('C6:G6')
        sheet_recibo.merge_cells('B7:G7')

        df.insert(loc=0, column='', value= None)
        del df['Mes']
        del df['Semana']
        for r in dataframe_to_rows(df, index=False, header=False):
            sheet_recibo.append(r)

        for i in range(12,30):
            sheet_recibo['G'+str(i)] = '=IF(E'+str(i)+'<>"",E'+str(i)+'*F'+str(i)+',"")'

        sheet_recibo['D31'] ='TOTAL DE HORAS'
        sheet_recibo['E31'] ='=SUM(E12:E30)'
        sheet_recibo['F31'] ='SON $'
        sheet_recibo['G31'] ='=SUM(G12:G30)'
        sheet_recibo['E36'] ='Firma:'
        sheet_recibo['E38'] ='Tipo y Nº Doc:'
        sheet_recibo['E40'] ='Aclaración:'

        sheet_recibo['D31'].font = font2
        sheet_recibo['E31'].font = font2
        sheet_recibo['G31'].font = font3
        sheet_recibo['F31'].font = font3
        sheet_recibo['G31'].fill = fill1
        sheet_recibo['F31'].fill = fill1

        sheet_recibo['B31'].border = border1
        sheet_recibo['C31'].border = border1
        sheet_recibo['D31'].border = border1
        sheet_recibo['E31'].border = border1
        sheet_recibo['G31'].border = border1
        sheet_recibo['F31'].border = border1

        sheet_recibo['E36'] ='Firma:'
        sheet_recibo['E40'] ='Aclaración:'

        sheet_recibo['F36'].border = border2
        sheet_recibo['F38'].border = border2
        sheet_recibo['F40'].border = border2

        sheet_recibo.merge_cells('F36:G36')
        sheet_recibo.merge_cells('F38:G38')
        sheet_recibo.merge_cells('F40:G40')

        wb2.save(fname)

        with output2:
            print('Agrego el recibo')
        
    else :
        print('Sin datos este mes')

In [15]:
button_month = widgets.Button(description="Corriente mes")
button_last_month = widgets.Button(description="Mes anterior")


button_month.on_click(get_this_month)
button_last_month.on_click(get_last_month)


widgets.HBox([
    button_last_month,
    button_month
    ],
    layout = widgets.Layout(width = '900px')
)

In [16]:
output2 = widgets.Output()

In [17]:
display(output2)

Output()